# Homework 5 - Visit the Wikipedia hyperlinks graph!
In this assignment we perform an analysis of the Wikipedia Hyperlink graph. In particular, given extra information about the categories to which an article belongs to, we are curious to rank the articles according to some criteria. 

<div style="text-align:center"><img src="https://i.pinimg.com/originals/a7/5f/dc/a75fdcab110ae11f155ed96f428a86ae.png"/> </div>

## Research questions


**[RQ1]** Build the graph <img src="https://latex.codecogs.com/gif.latex?G=(V,&space;E)" title="G=(V, E)" /> where *V* is the set of articles and *E* the hyperlinks among them, and provide its basic information:
 
- If it is direct or not
- The number of nodes
- The number of edges 
- The average node degree. Is the graph dense?

###### Build the graph!

In [29]:
from collections import defaultdict
import networkx as nx
from itertools import product
from collections import deque
import numpy as np

In [ ]:
file = open('wiki-topcats-reduced.txt','r').read().split('\n')
grafo = defaultdict(set)
for row in file:
        link=row.split('\t')
        try:
            grafo[link[0]].add(link[1])
            if link[1] not in grafo:
                grafo[link[1]] = set()
        except: 
            pass

###### Find out if it's directed or not:

We want to check if all the nodes that have edges coming form the node __62__ have an edge to the node __62__.

In [ ]:
print(all(["62" in grafo[edge] for edge in grafo['62']]))

As you can see, the statement above tells us that not all the nodes that are pointed by the node __62__ have an edge to the node __62__ and this is the counterexample to proof that our graph is directed.

###### Get the number of nodes!

In [ ]:
number_of_nodes=len(grafo)
number_of_nodes

###### Get the number of edges!

In [ ]:
number_of_edges= sum([len(grafo[node]) for node in grafo])
number_of_edges

###### Get the average node degree. Is the graph dense?

In graph theory, the degree (or valency) of a vertex of a graph is the number of edges incident to the vertex. The degree of a vertex $v$ is denoted $\deg(v)$.

In [ ]:
avg_degree= 2*number_of_edges/number_of_nodes
avg_degree

As we see, on average a node has 11-12 edges connected with him.
In mathematics, a dense graph is a graph in which the number of edges is close to the maximal number of edges, so
we can conclude that the graph is quietly dense.

## RQ2 
Given a category $C_0 = \{article_1, article_2, ... \}$ as input we want to rank all of the nodes in V according to the following criteria:

In [3]:
categories = defaultdict(list)
with open('wiki-topcats-categories.txt', 'r') as f:
    for row in f:
        splitted_row = row.split(' ')
        if len(splitted_row[1:]) > 3500:
            categories[splitted_row[0][9:-1]] = splitted_row[1:]

In [4]:
inverted_index = {}
inverted_index.update({nodo:categoria for categoria in categories for nodo in categories[categoria]})

In [5]:
G = nx.read_edgelist('wiki-topcats-reduced.txt', nodetype=str, delimiter='\t', create_using=nx.DiGraph())

In [6]:
for node in G:
    if node in inverted_index:
        G.nodes[node]['cat'] = inverted_index[node]

In [53]:
C0 = 'American_Jews'
C1 = 'English_television_actors'

In [8]:
# graph is a networkx directed graph
# source is the source node represented as a string.
# target is the target, same as before.

def shortest_path(graph, source, target):
    if not(graph.has_node(source) or graph.has_node(target)):
        return []
    visited = dict()#is the list containing the nodes of the shortest path between the source and the target
    path = []
    visited[source] = 'null'
    to_visit = deque([source])
    while(to_visit):
        visiting = to_visit.pop()
        vicini = set(G.neighbors(visiting))
        visited.update({vicino : visiting for vicino in vicini if vicino not in visited})
        to_visit.extendleft(vicini)
        if target in vicini:
            chiave = target
            path.append(target)
            while visited[chiave] != 'null':
                path.append(visited[chiave])
                chiave = visited[chiave]
            return path[::-1]

        
def spanning_tree(graph, source):
    visited = dict()#is the list containing the nodes of the shortest path between the source and the target
    visited[source] = 'null'
    to_visit = deque([source])
    while(to_visit):
        visiting = to_visit.pop()
        vicini = set(G.neighbors(visiting))
        yield {vicino : visiting for vicino in vicini if vicino not in visited}
        visited.update({vicino : visiting for vicino in vicini if vicino not in visited})
        to_visit.extendleft(vicini)
    

In [48]:
st = spanning_tree(G,'52')
depth = 0
source = '52'
#dest = '107'

class Generator:
    def __init__(self, gen):
        self.gen = gen

    def __iter__(self):
        self.value = yield from self.gen
        
        
def wrapper(G, source):
    def depth(st, source, dest, deep):
        for i in st:
            if dest == source:
                yield dest
                global var
                var = deep
                return var
            if dest in i:
                yield dest
                yield from depth(spanning_tree(G,source), source, i[dest], deep+1)
                return var
            
            
    st = spanning_tree(G, source)
    gen = []
    destinazionas = ['107']
    for dest in destinazionas:
        gen.append(Generator(depth(st, source ,dest,0)))

    return gen

In [49]:
v = list(wrapper(G, '52'))

In [38]:
def shortest_path_bidir(G, source, target):
    
    if source not in G or target not in G:
        return None

    result = pred_succ(G, source, target)
    if result is None:
        return None
    pred, succ, w = result
    # build path from pred+w+succ
    path = []
    # from source to w
    while w is not None:
        path.append(w)
        w = pred[w]
    path.reverse()
    # from w to target
    w = succ[path[-1]]
    while w is not None:
        path.append(w)
        w = succ[w]

    return path


def pred_succ(G, source, target):

    # does BFS from both source and target and meets in the middle
    if target == source:
        return ({target: None}, {source: None}, source)


    G_pred = G.pred
    G_succ = G.succ

    # predecesssor and successors in search
    pred = {source: None}
    succ = {target: None}

    # initialize borders, start with forward
    forward_border = [source]
    backward_border = [target]

    while forward_border and backward_border:
        if len(forward_border) <= len(backward_border):
            current_level = forward_border
            forward_border = []
            for v in current_level:
                for w in G_succ[v]:
                    if w not in pred:
                        forward_border.append(w)
                        pred[w] = v
                    if w in succ:  # path found
                        return pred, succ, w
        else:
            current_level = backward_border
            backward_border = []
            for v in current_level:
                for w in G_pred[v]:
                    if w not in succ:
                        succ[w] = v
                        backward_border.append(w)
                    if w in pred:  # found path
                        return pred, succ, w

    return None

In [39]:
shortest_path_bidir(G, '52', '32782')

['52', '1069112', '1061304', '1061148', '32782']

In [40]:
shortest_path_bidir(G, '52', '1163551')

['52', '1163551']

In [41]:
G.has_edge(u='1181401',v  ='107')

True

In [72]:
from time import time
pattese = []
start = time()
for nodec0 in categories[C0][0:5]:
    for nodec1 in categories[C1]:
        sp = shortest_path_bidir(G, nodec0, nodec1)
        if sp is not None:
            pattese.append([ sp ])
print(time()- start )

2.8150012493133545


In [73]:
pattese

[[['167', '279122', '824179', '430722', '1061148', '32782']],
 [['167', '279122', '1058611', '817692', '1054288', '40338']],
 [['167', '279122', '1163693', '1042988', '1044031', '40566']],
 [['167', '279122', '723026', '1502367', '1043362', '1043342', '53495']],
 [['167', '362517', '1184538', '1036284', '1039360', '72636']],
 [['167', '362517', '1184538', '1038249', '83294']],
 [['167',
   '279122',
   '824202',
   '817167',
   '1065973',
   '1040108',
   '92537',
   '92536']],
 [['167', '279122', '1061430', '1061598', '1044635', '94198']],
 [['167', '279122', '1058611', '1111340', '1041937', '30714', '96274']],
 [['167', '279122', '121190', '1060315', '1598208', '1598163', '109867']],
 [['167', '279122', '1058611', '749106', '113194']],
 [['167', '279122', '1400481', '1061740', '113259']],
 [['167', '279122', '1061501', '1061055', '131855']],
 [['167', '279122', '1025265', '1039555', '1179748', '131913']],
 [['167', '279122', '1400259', '1403869', '1034054', '134497']],
 [['167', '279